<a href="https://colab.research.google.com/github/antonypaulson/Detecting_interesting_VC_investments/blob/master/TCG_Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing necesscary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Defining a function to find subscription apps
def sub_finder(Y, column = 'description'):
    subs = []
    key_words = ['subscription', 'renewal', 'subscribe']
    for i in Y[column]:
        tokenized = nltk.word_tokenize(i)
        subscription = [word for word in tokenized if word.lower() in key_words]
        if len(subscription) > 0:
            subs.append(1)
        else:
            subs.append(0)
    return subs

#Creating a function to find the maximum rank jumps
def rank_growth(X):
    grow = []
    for index, row in X.iterrows():
        grth = np.max(row) - row[-3]
        grow.append(grth)
    return grow

#Creating a function to find the days the app stayed at the top rank
def top_rank_days(X):
    toprankdays = []
    for index, row in X.iterrows():
        mini = row.min(skipna = True)
        ndays = row.value_counts()[mini] 
        toprankdays.append(ndays)
    return toprankdays

In [0]:
#Importing the datasets
info_df = pd.read_pickle('/content/drive/My Drive/TCG/app_info_df.pkl')
rank_df = pd.read_pickle('/content/drive/My Drive/TCG/app_rank_df.pkl')

In [0]:
#Checking the info_df
info_df.head(2)

,app_name,developer,website,category,is_editor_choice,rating_oo5,num_ratings,has_iap,release_date,current_version,age_rating,file_size,editor_notes,description,os_compatibility,languages,price,itunes_link,date
itunes_app_id,,,,,,,,,,,,,,,,,,,
573984719,Advent Calendar: 25 Christmas Apps,MagicSolver,http://www.magicsolver.com/,Lifestyle,False,3.9,145,False,"Nov 24, 2015",1.1,Rated 4+,61.3 MB,NaN,\nDescription\n\n\nDiscover 25 of the finest f...,Requires iOS 7.0 or later. Compatible with iPh...,"English, Arabic, Dutch, French, German, Italia...",Free,https://itunes.apple.com/app/id573984719,2019-06-11-1512
424216726,Countdown‼,"Sevenlogics, Inc.",http://www.sevenlogics.com/mobile-apps/countdo...,Lifestyle,False,3.8,5500,True,"Jun 7, 2019",4.3.7,Rated 4+,152.7 MB,NaN,\nDescription\n\n\nCount down to your next eve...,Requires iOS 9.0 or later. Compatible with iPh...,"English, Danish, Dutch, Finnish, French, Germa...",Free,https://itunes.apple.com/app/id424216726,2019-06-11-1512


In [0]:
#Checking the dimensions of the info data frame
info_df.shape

(68834, 19)

In [0]:
#Checking for null values
info_df.isnull().sum()

app_name                0
developer               0
website               173
category                0
is_editor_choice        0
rating_oo5           9930
num_ratings          9930
has_iap                 0
release_date         3073
current_version      5779
age_rating              0
file_size               0
editor_notes        68087
description             0
os_compatibility        0
languages               0
price                   0
itunes_link             0
date                    0
dtype: int64

In [0]:
#Finding the top rank of each app
tr = rank_df.min(axis = 1, skipna = True)

In [0]:
#Getting the number of days the app remained at its best rank
trd = top_rank_days(rank_df)
#Checking the rank_df
rank_df.head(5)

date                                                        2016-12-08  ...  2019-06-11
itunes_app_id rating_list                                               ...            
1000008721    Lifestyle - topgrossingapplications                  NaN  ...         NaN
              Lifestyle - toppaidapplications                      NaN  ...         NaN
10000179      Health-and-fitness - topfreeapplications             NaN  ...         NaN
1000017994    Health-and-fitness - topfreeapplications             NaN  ...       125.0
              Health-and-fitness - topgrossingapplications         NaN  ...         NaN

[5 rows x 849 columns]

In [0]:
#Preparations for the Ranking Dataframe
#Performing the forward fill for null values
rank_df.fillna(method = 'ffill', axis = 1, inplace = True)
#Performing the backward fill for historical null values
rank_df.fillna(method = 'bfill', axis = 1, inplace = True)

In [0]:
#resetting the indexes of both data frames
info_df.reset_index(inplace = True)
rank_df.reset_index(inplace = True)

In [0]:
# #checking for patterns in null values
# num_null = []
# for i in range(len(rank_df)):
#      num_null.append(rank_df[rank_df.index == i].isnull().sum().sum())

# Adding derived columns to the Ranking data

In [0]:
#column for top rank
rank_df['top_rank'] = tr.reset_index()[0]
#column for days at top rank
rank_df['top_rank_days'] = trd

In [0]:
#column for tracking rank growth
rank_df['growth'] = rank_growth(rank_df[rank_df.columns[2:]])
#number of null values
#rank_df['num_null'] = num_null

In [0]:
#Creating a subscription service column
import nltk
nltk.download('punkt')
info_df['subscription_service'] = sub_finder(info_df, column = 'description')
n_subs = info_df['subscription_service'].value_counts()[1]
print (f'The number of subscription apps that were found was {n_subs}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The number of subscription apps that were found was 10707


In [0]:
#Getting all derived data into a seperate datframe
df_from_rank = rank_df[['itunes_app_id', 'rating_list', 'top_rank', 'top_rank_days', 'growth']]

In [0]:
#Creating the final dataframe by merging the subscription & rank dataframes together on 'itunes_app_id'
final_df = pd.merge(df_from_rank, info_df, on = 'itunes_app_id')

In [0]:
#This is how the final dataframe looks like
final_df.columns

Index(['itunes_app_id', 'rating_list', 'top_rank', 'top_rank_days', 'growth',
       'app_name', 'developer', 'website', 'category', 'is_editor_choice',
       'rating_oo5', 'num_ratings', 'has_iap', 'release_date',
       'current_version', 'age_rating', 'file_size', 'editor_notes',
       'description', 'os_compatibility', 'languages', 'price', 'itunes_link',
       'date', 'subscription_service'],
      dtype='object')

In [0]:
final_df.head(2)

,itunes_app_id,rating_list,top_rank,top_rank_days,growth,app_name,developer,website,category,is_editor_choice,rating_oo5,num_ratings,has_iap,release_date,current_version,age_rating,file_size,editor_notes,description,os_compatibility,languages,price,itunes_link,date,subscription_service
0,1000008721,Lifestyle - topgrossingapplications,166.0,1,0.0,DYC TongShu,Darren Gibbs,http://darrengibbs.info/TongShu,Lifestyle,False,5.0,6,False,09/07/2016,NaN,Rated 4+,10.1 MB,NaN,\nDescription\n\n\nThe Chinese “tong shu” or a...,Requires iOS 8.0 or later. Compatible with iPh...,English,$9.99,https://itunes.apple.com/app/id1000008721,2019-06-11-1512,0
1,1000008721,Lifestyle - toppaidapplications,83.0,1,41.0,DYC TongShu,Darren Gibbs,http://darrengibbs.info/TongShu,Lifestyle,False,5.0,6,False,09/07/2016,NaN,Rated 4+,10.1 MB,NaN,\nDescription\n\n\nThe Chinese “tong shu” or a...,Requires iOS 8.0 or later. Compatible with iPh...,English,$9.99,https://itunes.apple.com/app/id1000008721,2019-06-11-1512,0


In [0]:
final_df.to_csv('final_analysis_frame.csv')